In [6]:
import pybullet as p
import pybullet_data
import imageio
import numpy as np
import pickle
import PIL.Image
import os
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from IPython.display import HTML
print(os.getcwd(), "this is the current working directory")

/home/ubuntu/bayes3d/experiments/physics this is the current working directory


In [7]:
def display_video(frames, framerate=30):
    height, width, _ = frames[0].shape
    dpi = 70
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
    fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
    matplotlib.use(orig_backend)  # Switch back to the original backend.
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
      im.set_data(frame)
      return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                   interval=interval, blit=True, repeat=False)
    return HTML(anim.to_html5_video())

In [8]:
def object_pose_in_camera_frame(object_id, view_matrix):
    object_pos, object_orn = p.getBasePositionAndOrientation(object_id) # world frame
    world2cam = np.array(view_matrix).reshape([4,4]).T # world --> cam 
    object_transform_matrix = np.eye(4)
    object_transform_matrix[:3, :3] = np.reshape(p.getMatrixFromQuaternion(object_orn), (3, 3))
    object_transform_matrix[:3, 3] = object_pos
    return world2cam @ object_transform_matrix

In [75]:
# Initialize the PyBullet physics simulation
p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Set up the simulation environment
p.setGravity(0, 0, -10)
p.setTimeStep(1.0/240.0)
plane_id = p.loadURDF("plane.urdf")


occ_shape = p.createCollisionShape(shapeType=p.GEOM_MESH, fileName="assets/plane.obj", meshScale=[0.0667,0.0667,0.0667])
occ_id = p.createMultiBody(0, occ_shape, basePosition=[0,-1,1], baseOrientation = [ 0.7071068, 0, 0, 0.7071068 ])
# p.loadURDF("plane.urdf", globalScaling = 0.01, basePosition=[0,-1,1], baseOrientation = [ 0.7071068, 0, 0, 0.7071068 ])



# Set the friction coefficient for the plane
friction_coefficient = 0.0  # Adjust this value as needed
p.changeDynamics(plane_id, -1, lateralFriction=friction_coefficient)

# Create the first box
box_radius = 0.5
box_mass = 1
box_position = [-2.5, 0, 1.5]
box_start_velocity = [4, 0, 0]
mesh_scale = [0.5,0.5,0.5]
box_shape = p.createCollisionShape(shapeType=p.GEOM_MESH, fileName="assets/cube.obj", meshScale=mesh_scale)
box_id = p.createMultiBody(box_mass, box_shape, basePosition=box_position)

p.resetBaseVelocity(box_id, box_start_velocity) #, [0, 10, 0])

p.changeDynamics(box_id, -1, restitution = 1)
object_id = box_id

# Arrays for serialization 
frames = []
box_poses   = []
occ_poses = []
depths = []

w = 480
h = 360

# to get size of an id
aabb_min, aabb_max = p.getAABB(occ_id)
aabb_dimensions = [aabb_max[i] - aabb_min[i] for i in range(3)]
print(aabb_dimensions)


# Step through the simulation
for i in range(360):
    p.stepSimulation()

    if i%4 ==0:
        # record positions of boxs

        view_matrix = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0, 0, 0], distance=5, yaw=0, pitch=-10, roll=0,
                                                        upAxisIndex=2)
        proj_matrix = p.computeProjectionMatrixFOV(fov=60, aspect=float(w) / h, nearVal=0.1, farVal=100.0)

        (_, _, px, d, _) = p.getCameraImage(width=w, height=h, viewMatrix=view_matrix,
                                            projectionMatrix=proj_matrix, renderer=p.ER_BULLET_HARDWARE_OPENGL)
        rgb_array = np.array(px, dtype=np.uint8)
        rgb_array = np.reshape(rgb_array, (h, w, 4))
        rgb_array = rgb_array[:, :, :3]  # remove alpha channel
        depths.append(np.array(d))
        frames.append(rgb_array)

        box_poses.append(object_pose_in_camera_frame(object_id, view_matrix))

plane_cam_pose = object_pose_in_camera_frame(plane_id, view_matrix)
occ_pose = object_pose_in_camera_frame(occ_id, view_matrix)

p.disconnect()


[2.0049999999999994, 0.004000000000000559, 2.004999999999999]


In [76]:
display_video(frames, framerate=60)

In [79]:
array_dict = {
    'box': box_poses,
    'plane': plane_cam_pose,
    'occ' : occ_pose
}

np.savez('poses_camera_frame.npz', **array_dict)


error: Not connected to physics server.

In [63]:
pybullet_data.getDataPath()

'/opt/conda/envs/bayes3d/lib/python3.9/site-packages/pybullet_data'